In [30]:
import re
import os
import pickle as pkl

slice_ = 4000

In [31]:
runs = [9, 10, 11]
methods = ['random', 'state', 'trace']
bug_types = ['Large LogIndex: %d']
slice_ = 5900
for run in runs:
    for method in methods:
        print(f'{method} run: {run}')
        basedir = f'experiments_berkay/raft_3_3/{method}/output{run}/'
        with open(os.path.join(basedir, f'saved/{method}_stats.pkl'), 'rb') as f:
            stats = pkl.load(f)
        print(f'Length: {len(stats["coverage"])}')
        print(f'Coverage: {stats["coverage"][:slice_][-1]}')
        print(f'Len bug_iters: {len(stats["bug_iterations"])}')
        print(f'Bug_iters:')
        print(stats["bug_iterations"])
        print('-------------------------')


random run: 9
Length: 6081
Coverage: 4023
Len bug_iters: 60
Bug_iters:
[64, 128, 192, 216, 221, 460, 585, 1074, 1235, 1394, 1612, 1720, 1927, 2004, 2052, 2119, 2191, 2272, 2277, 2288, 2304, 2481, 2497, 2613, 2720, 2759, 2769, 2889, 2893, 2906, 2923, 3045, 3181, 3202, 3243, 3540, 3613, 3802, 3837, 3841, 3842, 3861, 3906, 3992, 4241, 4611, 4650, 4816, 4936, 4979, 5079, 5168, 5181, 5591, 5616, 5634, 5681, 5726, 5779, 5959]
-------------------------
state run: 9
Length: 6341
Coverage: 4275
Len bug_iters: 49
Bug_iters:
[10, 58, 61, 189, 238, 353, 362, 604, 1014, 1090, 1112, 1201, 1243, 1307, 1323, 1512, 1734, 1748, 2011, 2071, 2442, 2844, 3019, 3112, 3243, 3248, 3577, 3661, 3826, 3986, 3987, 4109, 4228, 4338, 4549, 4628, 4661, 4903, 4913, 4920, 4983, 5046, 5147, 5201, 5300, 5549, 5874, 5960, 6123]
-------------------------
trace run: 9
Length: 6931
Coverage: 3643
Len bug_iters: 36
Bug_iters:
[202, 503, 515, 586, 602, 653, 660, 683, 852, 1060, 1245, 1255, 1450, 2174, 2592, 2699, 2834, 3245, 

In [34]:
all_bugs = {}
unexpected_bugs = {}
#r'\b\d+\b'
slice_ = 4000
for run in runs:
    for method in methods:
        bugs = []
        if method not in all_bugs.keys():
            all_bugs[method] = []
        if method not in unexpected_bugs.keys():
            unexpected_bugs[method] = []
        # print(f'{method} run: {run}')
        basedir = f'experiments_berkay/raft_3_3/{method}/output{run}/errors/'
        for file in os.listdir(basedir):
            iter = [int(n) for n in re.findall(r'\d+', file)][0]
            with open(f'{basedir}{file}/stderr.log', 'r') as f:
                lines = f.readlines()
            if 'Large LogIndex:' in lines[0]:
                li = [int(n) for n in re.findall(r'\b\d+\b', lines[0])]
                if li[0] < 4:
                    continue
                all_bugs[method].append((run, iter, li[0]))
            elif 'Negative LogIndex!' in lines[0]:
                all_bugs[method].append((run, iter, -1))
            else:
                unexpected_bugs[method].append((method, run, iter))
print(all_bugs)
print(unexpected_bugs)



{'random': [(9, 5634, 4), (9, 64, 5), (9, 5959, 4), (9, 2052, 4), (10, 1712, 4), (10, 3854, 5), (10, 2890, 4), (10, 2837, 4), (10, 1248, 4), (11, 1573, 4)], 'state': [(9, 4628, 4), (9, 5874, 4), (10, 3366, 4), (10, 2994, 4), (11, 2980, 4), (11, 3882, 5), (11, 1106, 4), (11, 806, 4)], 'trace': [(9, 5505, 4), (9, 852, 4), (10, 274, 4), (10, 6695, 4), (10, 4310, 4), (11, 4842, 4), (11, 4217, 4), (11, 1905, 4), (11, 1919, 5)]}
{'random': [('random', 9, 3613), ('random', 10, 5294), ('random', 10, 5363), ('random', 10, 394), ('random', 11, 2786), ('random', 11, 3447), ('random', 11, 3499), ('random', 11, 3439)], 'state': [('state', 9, 6343), ('state', 9, 3987), ('state', 9, 3826), ('state', 10, 340), ('state', 10, 3105), ('state', 10, 2708), ('state', 11, 1024), ('state', 11, 1659), ('state', 11, 5402), ('state', 11, 3502)], 'trace': [('trace', 9, 1060), ('trace', 10, 2018), ('trace', 10, 3479), ('trace', 11, 3737)]}
